In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt

# Reading Data


In [2]:
df_2021 = pd.read_csv('data/2021.csv')
# df_2021.dropna(inplace=True)
# df_2021.tail(20)

df_2020 = pd.read_csv('data/2020.csv')
# df_2020.dropna(inplace=True)
# df_2020.tail(20)

treatment_definitions = pd.read_csv('data/Plough & Till Plan/Definitions.csv')
del treatment_definitions['Unnamed: 3']
del treatment_definitions['Unnamed: 4']

treatment_definitions.head(20)

,Plot,Code,Description
0,1,PLPHMNC1P,Plough low pressure high moisture no cover cro...
1,2,PHPLMCC3P,Plough high pressure low moisture cover crop 3...
2,3,PLPHMNC3P,Plough low pressure high moisture no cover cro...
3,4,PHPLMNC1P,Plough high pressure low moisture no cover cro...
4,5,PLPLMNC3P,Plough low pressure low moisture no cover crop...
5,6,PHPLMNC3P,Plough high pressure low moisture no cover cro...
6,7,PLPLMNC1P,Plough low pressure low moisture no cover crop...
7,8,PHPHMNC1P,Plough high pressure high moisture no cover cr...
8,9,PHPHMCC1P,Plough high pressure high moisture cover crop ...
9,10,PLPHMCC1P,Plough low pressure high moisture cover crop 1...


### Setting up data for models & testing

In [3]:
df_2021['ploughed'] = 0
df_2021['tilled'] = 0
df_2021['high pressure'] = 0
df_2021['low pressure'] = 0
df_2021['high moisture'] = 0
df_2021['low moisture'] = 0
df_2021['covercrop'] = 0
df_2021['no covercrop'] = 0
df_2021['high traffic'] = 0
df_2021['low traffic'] = 0

for index, row in treatment_definitions.iterrows():
    if 'Plough' in row['Description']:
        df_2021.at[index, 'ploughed'] = 1
    if 'Min Till' in row['Description']:
        df_2021.at[index, 'tilled'] = 1
    if 'low pressure' in row['Description']:
        df_2021.at[index, 'low pressure'] = 1
    if 'high pressure' in row['Description']:
        df_2021.at[index, 'high pressure'] = 1
    if 'low moisture' in row['Description']:
        df_2021.at[index, 'low moisture'] = 1
    if 'high moisture' in row['Description']:
        df_2021.at[index, 'high moisture'] = 1
    if '3 Pass' in row['Description']:
        df_2021.at[index, 'high traffic'] = 1
    if '1 Pass' in row['Description']:
        df_2021.at[index, 'low traffic'] = 1

    # Cover crop is a special case, because "cover crop" is also a substring of "no cover crop"
    # we need to check "no cover crop" first, then if not found set 'covercrop' as true
    if 'no cover crop' in row['Description']:
        df_2021.at[index, 'no covercrop'] = 1
    else:
        df_2021.at[index, 'covercrop'] = 1

df_2021.tail(30)

,Plot,Treatment,BD 0-10cm,BD 10-20cm,BD 20-30cm,BD Moisture 0-10cm,BD Moisture 10-20 cm,BD Moisture 20-30 cm,Emergence,Chlorophyll 28th May,...,ploughed,tilled,high pressure,low pressure,high moisture,low moisture,covercrop,no covercrop,high traffic,low traffic
66,67,PLPHMNCHA,1.36,1.40,1.64,17.3,21.7,19.6,7.2,43.6,...,1,0,0,1,1,0,0,1,1,0
67,68,PHPHMCCHA,1.30,1.39,1.57,18.1,26.2,25.7,5.6,42.6,...,1,0,1,0,1,0,1,0,1,0
68,69,PHPLMNCHA,1.25,1.53,1.58,17.6,28.8,26.6,8.2,36.2,...,1,0,1,0,0,1,0,1,1,0
69,70,PHPHMNCHA,1.28,1.09,1.62,18.0,19.2,19.9,8.2,42.9,...,1,0,1,0,1,0,0,1,1,0
70,71,PLPLMNCLA,1.22,1.40,1.70,19.2,24.8,22.3,7.2,41.4,...,1,0,0,1,0,1,0,1,0,1
71,72,PHPLMNCLA,1.08,1.38,1.46,19.9,28.5,28.6,7.6,40.5,...,1,0,1,0,0,1,0,1,0,1
72,73,PHPHMNCLA,1.25,1.32,1.40,17.5,26.8,30.2,8.2,37.2,...,1,0,1,0,1,0,0,1,0,1
73,74,PLPHMNCLA,1.11,1.25,1.36,15.5,24.2,30.1,7.2,40.1,...,1,0,0,1,1,0,0,1,0,1
74,75,PHPHMCCLA,1.20,1.40,1.55,15.0,27.6,27.0,7.2,41.0,...,1,0,1,0,1,0,1,0,0,1
75,76,PHPLMCCHA,1.07,1.60,1.63,17.2,28.6,28.2,8.2,41.6,...,1,0,1,0,0,1,1,0,1,0


In [4]:
# Setting targets and removing them from dataframe
y = df_2021['Combine Yield (t/ac)']
y = y.fillna(0)
del df_2021['Combine Yield (t/ac)']

emergenceY = df_2021['Emergence']
emergenceY = emergenceY.fillna(0)
del df_2021['Emergence']

# Features
X = df_2021[['ploughed', 'tilled', 'high pressure', 'low pressure', 'high moisture', 'low moisture', 'covercrop', 'no covercrop', 'high traffic', 'low traffic']]
X = X.fillna(0)
X.tail()

,ploughed,tilled,high pressure,low pressure,high moisture,low moisture,covercrop,no covercrop,high traffic,low traffic
91,0,1,0,1,0,1,0,1,0,1
92,0,1,0,1,1,0,0,1,0,1
93,0,1,1,0,1,0,1,0,1,0
94,0,1,1,0,1,0,1,0,0,1
95,0,1,0,1,0,1,1,0,0,1


In [5]:
# High/low pressure - tyre / bulk density
# Flask

In [6]:
# Splitting data for training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_emergence, X_test_emergence, y_train_emergence, y_test_emergence = train_test_split(X, emergenceY, test_size=0.33, random_state=42) # Emergence

## Linear Regression Model

In [7]:
# Linear Regression Model
reg = LinearRegression().fit(X_train, y_train)
reg.score(X, y) # score

0.6067229143280584

In [8]:
# Linear Regression Model
regEmergence = LinearRegression().fit(X_train_emergence, y_train_emergence)
regEmergence.score(X, emergenceY) # score

0.17624316756640968

## SVR Model

In [9]:
# SVR Model
svr = SVR().fit(X_train, y_train)
svr.score(X, y) # score

0.711948683294384

In [10]:
# SVR Model
svrEmergence = SVR().fit(X_train_emergence, y_train_emergence)
svrEmergence.score(X, emergenceY) # score

0.21494329539106138

In [11]:
# evaluate the model
yhat = reg.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 0.418


## Testing Predictions

In [12]:
# Plot 36
yieldPredict = reg.predict(np.array([[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]])) # Actual === 2.8
emergencePredict = regEmergence.predict(np.array([[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]])) # Actual === 6.0
yieldPredict

/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([2.56393444])

In [13]:
# Plot 55
yieldPredict = reg.predict(np.array([[0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]])) # Actual === 0.24
emergencePredict = regEmergence.predict(np.array([[0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]])) # Actual === 8.2
yieldPredict

/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([0.61583164])

### SVR Predictions

In [14]:
# Plot 36
yieldPredict = svr.predict(np.array([[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]])) # Actual === 2.8
yieldPredict

/Users/ciaranflanagan/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


array([2.29365297])